In [3]:
from hyperspectral import *
import numpy as np
import pickle

correspondences = pickle.load(open('data/training_correspondences.p', 'rb'))
hsi_import = np.load('data/hsi_data.npz')
hsi_data = hsi_import['hyperspectral_data']
wavelengths = hsi_import['wavelengths']

#### Compile Train / Test Data from Correspondences ####

In [4]:
hsi_data_matrix = multispectral_raster_to_matrix(hsi_data)
image_width = hsi_data.shape[1]

observation_indices = np.zeros(len(correspondences), dtype=np.int32)
min_target_indices = np.zeros(len(correspondences), dtype=np.int32)
med_target_indices = np.zeros(len(correspondences), dtype=np.int32)
max_target_indices = np.zeros(len(correspondences), dtype=np.int32)

# compile index arrays pointing to rows in the data matrix for pixel correspondences

for pixel, index in zip(correspondences.keys(), np.arange(len(correspondences))):

    observation_indices[index] = pixel[0] * image_width + pixel[1]
    
    # compute spectra norms for all corresponding pixels; select the min, median, and max
    open_norms = [ np.linalg.norm(get_spectrum(hsi_data, px)) for px in correspondences[pixel] ]
    norm_indices = np.argsort(open_norms)
    min_norm_pixel = correspondences[pixel][norm_indices[0]]
    med_norm_pixel = correspondences[pixel][norm_indices[len(norm_indices)//2]]
    max_norm_pixel = correspondences[pixel][norm_indices[-1]]
    
    min_target_indices[index] = min_norm_pixel[0] * image_width + min_norm_pixel[1]
    med_target_indices[index] = med_norm_pixel[0] * image_width + med_norm_pixel[1]
    max_target_indices[index] = max_norm_pixel[0] * image_width + max_norm_pixel[1]


# create randomly shuffled train/test split of the compiled observations

training_percentage = 0.8
selection_indices = np.arange(len(correspondences), dtype=np.int32)
np.random.shuffle(selection_indices)
cutoff = int(training_percentage * len(correspondences))
training_indices = selection_indices[:cutoff]
testing_indices = selection_indices[cutoff:]

training_observations = hsi_data_matrix[observation_indices[training_indices], :]
training_targets = { 'min' : hsi_data_matrix[min_target_indices[training_indices], :], 
                   'median' : hsi_data_matrix[med_target_indices[training_indices], :],
                   'max' : hsi_data_matrix[max_target_indices[training_indices], :] }

test_observations = hsi_data_matrix[observation_indices[testing_indices], :]
test_targets = { 'min' : hsi_data_matrix[min_target_indices[testing_indices], :], 
                   'median' : hsi_data_matrix[med_target_indices[testing_indices], :],
                   'max' : hsi_data_matrix[max_target_indices[testing_indices], :] }

#### Evaluate Various Regression Models ####

In [10]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.kernel_ridge import KernelRidge
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR

pipelines = {
    'full_ridge' : Pipeline(steps=[('ridge', KernelRidge())]),
    'pca_ridge' : Pipeline(steps=[('pca', PCA()), ('ridge', KernelRidge())]),
    'full_svr' : Pipeline(steps=[('svr', MultiOutputRegressor(SVR()))]),
    'pca_svr' : Pipeline(steps=[('pca', PCA()), ('svr', MultiOutputRegressor(SVR()))])
}

parameters = {
    'full_ridge' : { 'ridge__alpha': [ 10.0, 1.0, 0.1, 0.01 ], 'ridge__gamma': [ 1.0, 5.0, 10.0  ], 
                    'ridge__degree' : [ 3, 5 ], 'ridge__kernel' : ['polynomial', 'rbf'] },
    'pca_ridge' : { 'pca__n_components': [ 10, 25, 50 ], 'ridge__alpha': [ 10.0, 1.0, 0.1, 0.01 ], 'ridge__gamma': [ 1.0, 5.0, 10.0  ], 
                    'ridge__degree' : [ 3, 5 ], 'ridge__kernel' : ['polynomial', 'rbf'] },
    'full_svr' : { 'svr__estimator__C': [ 0.5, 0.75, 1.0, 1.25 ], 'svr__estimator__gamma': [ 0.001, 0.005, 0.01, 0.05  ] },
    'pca_svr' : { 'pca__n_components': [ 10, 25, 50 ], 'svr__estimator__C': [ 0.5, 0.75, 1.0, 1.25 ], 'svr__estimator__gamma': [ 0.001, 0.005, 0.01, 0.05  ] }
}

best_params = {}

for pipe in pipelines:
    
    model = GridSearchCV(pipelines[pipe], parameters[pipe], scoring='neg_root_mean_squared_error', cv=10, n_jobs=4, refit=True)
    for target in training_targets:

        print('\nEvaluating the', pipe, 'pipeline using correspondences with the', target, 'norm target spectra.')
        
        model.fit(training_observations, training_targets[target])
        print('Selected Model Parameters:', model.best_params_)
        print('\nCross Validated Average RMSE: {:.3f}'.format(-model.best_score_))
    
        # as refit=True, the search will have retrained on all training data and can be used to make predictions
        predicted_targets = model.predict(test_observations)
        print('Testing Average RMSE: {:.3f}'.format(np.mean(np.sqrt(mean_squared_error(test_targets[target], predicted_targets)))))


Evaluating the full_ridge pipeline using correspondences with the min norm target spectra.
Selected Model Parameters: {'ridge__alpha': 0.1, 'ridge__degree': 3, 'ridge__gamma': 5.0, 'ridge__kernel': 'rbf'}

Cross Validated Average RMSE: 0.030
Testing Average RMSE: 0.032

Evaluating the full_ridge pipeline using correspondences with the median norm target spectra.
Selected Model Parameters: {'ridge__alpha': 0.1, 'ridge__degree': 3, 'ridge__gamma': 5.0, 'ridge__kernel': 'rbf'}

Cross Validated Average RMSE: 0.031
Testing Average RMSE: 0.033

Evaluating the full_ridge pipeline using correspondences with the max norm target spectra.
Selected Model Parameters: {'ridge__alpha': 0.1, 'ridge__degree': 3, 'ridge__gamma': 5.0, 'ridge__kernel': 'rbf'}

Cross Validated Average RMSE: 0.040
Testing Average RMSE: 0.050

Evaluating the pca_ridge pipeline using correspondences with the min norm target spectra.
Selected Model Parameters: {'pca__n_components': 10, 'ridge__alpha': 0.1, 'ridge__degree': 3,

#### Train and Export Best Model ####

In [12]:
final_observations = np.vstack((training_observations, test_observations))
final_targets = np.vstack((training_targets['min'], test_targets['min']))
final_model = KernelRidge(alpha=0.1, gamma=5.0, kernel='rbf')
final_model.fit(final_observations, final_targets)

output = { 'observations': final_observations, 'targets': final_targets, 'model': final_model }
pickle.dump(output, open('data/trained_regressor.p', 'wb'))